In [2]:
PATHSAVE="D:/DISSERTATION/newBACKUP/labeling/"
# google drive specific pytorch instructions if you want to run on google collab gpu
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
# !pip3 install torchvision

In [0]:
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from PIL import Image
import torch,numpy as np,torch.nn as nn,torch.nn.functional as F,torch.optim as optim,torchvision.models as models,torch.backends.cudnn as cudnn,torchvision.datasets as datasets,cv2,time,os,shutil,sys
import torch

In [1]:
# drive access
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# switch color format: dataRGB,dataHSV,dataGray,dataEdge
os.listdir('/My Drive/PublicationData/dataRGB/')
# files.download('example.txt')

In [0]:
def validate(val_loader,trainYes):
    correct = 0
    total = 0
    results=[]
    with torch.no_grad():
        for data in val_loader:
            images, cpuLabels = data
            
            # can change if on cpu
            images, labels=images.cuda(device), cpuLabels.cuda(device)

            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)
            subRes=(predicted == labels).sum().item()

            if trainYes: # for validation batch
                print(subRes/50,end=", ")
            results.append(subRes/50)
            total += labels.size(0) 
            correct += subRes

    print('total: %d %%' % (
        100 * correct / total))
    return 100 * correct / total,results

def test_set(test_loader):
    correct = 0
    total = 0
    results=[]
    with torch.no_grad():
        for data in test_loader:
            images, cpuLabels = data
            if(torch.cuda.is_available()):
                images, labels=images.cuda(device), cpuLabels.cuda(device)# IMPORTANT
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            subRes=(predicted == labels).sum().item()
            
            total += labels.size(0)
            correct += subRes 

    res=100 * correct / total
    return res

In [5]:
print("started")
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])

transformations=transforms.Compose([
  transforms.RandomResizedCrop(224),
  transforms.ToTensor(),
  normalize,
])
transformations2=transforms.Compose([
  transforms.RandomCrop(224),
  transforms.ToTensor(),
  normalize,
])


started


In [0]:
root_path = 'drive/My Drive/PublicationData/data' 
testtype="RGB" # dataRGB,dataHSV,dataGray,dataEdge
train_loader = torch.utils.data.DataLoader(
  datasets.ImageFolder(root_path+testtype+'/train/', transformations),
  batch_size=32, shuffle=True,
  num_workers=5, pin_memory=True)

# can uncomment while training.
# val_loader = torch.utils.data.DataLoader(
#     datasets.ImageFolder(root_path+testtype+'/val/', transformations2),
#     batch_size=50, shuffle=False,#50 10,5   32-10
#     num_workers=5, pin_memory=True)

# comment while testing.
test_loader = torch.utils.data.DataLoader(
  datasets.ImageFolder(root_path+testtype+'/test/', transformations2),
  batch_size=116, shuffle=False,
  num_workers=5, pin_memory=True)

device = torch.device("cuda") #IMPORTANT
learning_rate = 0.01

criterion = nn.CrossEntropyLoss().cuda()

In [7]:
print(torch.cuda.is_available())

True


In [9]:
for trial in range(10):# epoch
    # model=models.resnet18(pretrained=True)
    model=models.resnet34(pretrained=True)
    #   model=torch.load("PATH2model/model.m")

    print("trial",trial)
    if(torch.cuda.is_available()):
        model = model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(50):
        print("epoch",epoch)
        model.train()
        running_loss = 0.0

        for i, data in enumerate(train_loader, 0):

            # get the inputs
            inputs, labels = data
            if(torch.cuda.is_available()):
            inputs, labels=inputs.cuda(device), labels.cuda(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs,labels)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print('tr loss %d: %.3f' %(epoch + 1, running_loss / len(train_loader)))
        running_loss = 0.0


        model.eval()
        
        #training acc:
        print("training Acc:",round(test_set(train_loader),2))
        
        # validation
        # validate(train_loader,False)
        # print("validating:",end=" ")
        
        # testing
        testRes=test_set(test_loader)
        print("test Acc:",round(testRes,2))
        
        # model save
        torch.save(model.state_dict(), root_path+testtype+"/PublicationModel/"+trial+"state_ResNet34_True_"+testtype+"_t"+str(trial)+"_50_001_"+str(epoch)+"_"+str(testRes)+"p.m")
  

eeee 5
epoch 0
tr loss 1: 1.568
training Acc: 31.31
test Acc: 28.97
epoch 1
tr loss 2: 1.268
training Acc: 45.46
test Acc: 42.59
epoch 2
tr loss 3: 1.171
training Acc: 41.0
test Acc: 20.34
epoch 3
tr loss 4: 1.084
training Acc: 54.78
test Acc: 46.03
epoch 4
tr loss 5: 1.011
training Acc: 56.07
test Acc: 50.34
epoch 5
tr loss 6: 0.962
training Acc: 60.04
test Acc: 58.28
epoch 6
tr loss 7: 0.908
training Acc: 62.7
test Acc: 59.14
epoch 7
tr loss 8: 0.864
training Acc: 62.35
test Acc: 50.34
epoch 8
tr loss 9: 0.842
training Acc: 67.7
test Acc: 62.24
epoch 9
tr loss 10: 0.806
training Acc: 68.36
test Acc: 72.76
epoch 10
tr loss 11: 0.777
training Acc: 63.44
test Acc: 50.86
epoch 11
tr loss 12: 0.751
training Acc: 65.37
test Acc: 65.52
epoch 12
tr loss 13: 0.734
training Acc: 68.03
test Acc: 70.52
epoch 13
training Acc: 69.45
test Acc: 76.21
epoch 14
tr loss 15: 0.704
training Acc: 70.29
test Acc: 64.14
epoch 15
tr loss 16: 0.691
training Acc: 74.99
test Acc: 75.52
epoch 16
tr loss 17: 0.65